Upewnij się, że jesteś w odrębnym środowisku od środowiska używanego do konwersji modelu na model TFJS


Ten plik pokazuje krok po kroku w jaki sposób była trenowana ta sieć. W kodzie wprowadziłem stosowne komentarze po angielsku.

Najpierw są pobierane odpowiednie biblioteki oraz ustawiony seed dla maksymalnej reprodukowalności badań.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.losses import BinaryCrossentropy
from keras import layers
import tensorflow_hub as hub
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, matthews_corrcoef

#In order to main reproducibility
tf.keras.utils.set_random_seed(42)
# Check if you have properly installed GPU support
# tf.config.list_physical_devices()

Teraz ustawiane są 3 zmienne:
- jakie punkty chcemy by model brał pod uwagę.
- z ilu powtórzeń trenowania modelu chcemy liczyć średnią metryk.
- jaki jest próg klasyfikacji klas na pozytywną lub negatywną. Powyżej - pozytywna. Poniżej - negatywna.

In [ ]:
keypoints_configuration={
    'all': 17,
    'head_shoulders': 7,
    'head_shoulders_hands': 11,

}

#How many keypoints we want to use
choose_configuration='all'
keypoints_amount=keypoints_configuration[choose_configuration]

#How many times repeat fitting the model in order to get average of metrics
#set 1 or 2 when want to quickly check if everything works
repeat=10

#Set threshold when evaluating the model. Above it - improper posture, positive class. 
#Below it - proper posture, negative class
threshold=0.5

Załadowanie zdjęć/Datasetu. Domyślnie ładuje się gotowy zbiór punktów.

Uwaga! Jeśli chcesz korzystać z własnego datasetu zdjęć proszę upewnić się, że:
- rozpakowano pliki odpowiednio (wykorzystaj ```extract_files.ipynb```).
- odkomentowano poniższy kod, a zakomentowano 2 pierwsze linijki
- ustawiano zmienną ```samples_path``` tak by wskazywała na folder z 2 folderami zawierającymi zdjęcia z klasami. Domyślnie ustawia "classified_images"
- nazwa folderu ze zdjęciami z nieodpowiednią posturą (klasa pozytywna) alfabetycznie jest druga od nazwy folderu ze zdjęciami z odpowiednią posturą (klasa negatywna). Można umieścić zdjęcia bezpośrednio do odpowiednich folderów.

Program trenujący model na zdjęciach w trakcie przygotowywania mojej pracy zabierał ok. 40 minut czasu, więc w przypadku umieszczenia własnego, o podobnej skali datasetu, należy się spodziewać dłuższego oczekiwania. przeciwnym razie skorzystaj z gotowego datasetu, który zawiera od razu zdjęcia przekonwertowane na punkty.

In [ ]:
dataset_path="../ready_components/keypoints_dataset/"
keypoints_dataset = tf.data.Dataset.load(dataset_path)
"""
samples_path = "./classified_images/"
#Initialize needed variables

model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']
#Set source of samples

#Important note: your positive class should be in a folder which is alphabetically second, otherwise 
#it will be treated as a negative class, which is not what we want
image_dataset = keras.utils.image_dataset_from_directory(
    directory=samples_path,
    labels='inferred',
    label_mode='binary',
    batch_size=None,
    shuffle=True,
    image_size=(256, 256),
    seed=42)

def translate_into_points(image):
    image = tf.expand_dims(image, axis=0)
    image = tf.cast(image, dtype=tf.int32)
    output=tf.squeeze(movenet(image)['output_0'])
    return output;


def map_into(filepath, label):
    return translate_into_points(filepath), label

keypoints_dataset=image_dataset.map(map_into)
print(keypoints_dataset)
"""


Nieinteresujące punkty są eliminowane z datasetu.

In [ ]:

if(choose_configuration != 'all'):
    def extract_from_head_to_wrists(keypoints, label):
        keypoints = keypoints[:keypoints_amount]
        return keypoints, label

    keypoints_dataset = keypoints_dataset.map(extract_from_head_to_wrists)


In [ ]:
print(len(keypoints_dataset))
keypoints_dataset

Dataset jest dzielony na część treningową, walidacyjną i testową w proporcjach 60/20/20

In [ ]:
#Warning! This step below can get above 30min to complete if using custom dataset!

#Making 60/20/20 split
train_dataset, test_dataset = tf.keras.utils.split_dataset(keypoints_dataset, left_size=0.8, shuffle=True, seed=42)
train_dataset, validation_dataset = keras.utils.split_dataset(train_dataset, right_size=0.25, shuffle=True, seed=42)
print(len(train_dataset))
print(len(test_dataset))
print(len(validation_dataset))
train_dataset = train_dataset.batch(100)
test_dataset = test_dataset.batch(100)
validation_dataset = validation_dataset.batch(100)

Utworzony jest model klasyfikujący. Hiperparametr learning rate wynosi 0.001.

In [ ]:

def create_and_compile_model():
    model = keras.Sequential(
        [
            keras.Input(shape=(keypoints_amount,3,)),
            layers.Flatten(),
            layers.Dense(128, activation="relu"),
            layers.Dropout(0.5),
            layers.Dense(64, activation="relu"),
            layers.Dropout(0.5),
            layers.Dense(1, activation='sigmoid'),
        ]
    )
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
        )
    return model

model = create_and_compile_model()

model.summary()

Poniżej zachodzi trenowanie modelu. Znaczna część poniższego bloku służy obliczeniu metryk.

In [ ]:

#Get average of 5 runs:
metrics_values={
    'loss':[],
    'MCC':[],
    'accuracy':[],

    'confusion_matrix':[],

    'negative_precision':[],
    'negative_recall':[],
    'negative_f1':[],
    'positive_precision':[],
    'positive_recall':[],
    'positive_f1':[],

}

earlystopping = keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', 
                                              patience=10, verbose=1,)
for i in range(repeat):
    model = create_and_compile_model()
    history = model.fit(train_dataset, 
                    epochs=200,
                    validation_data=validation_dataset,
                    callbacks=[earlystopping],        
         )
    raw_y_pred = model.predict(test_dataset) #get predictions
    raw_y_true = np.array(list(test_dataset.unbatch().map(lambda keypoints, label: label).as_numpy_iterator())) #get true labels
    #Convert predictions into True or False
    y_pred = tf.math.greater(raw_y_pred,threshold)
    #Quickly convert 1 or 0 to True or False for true labels
    y_true = tf.math.greater(raw_y_true, 0.5)

    metrics_values['MCC'].append(matthews_corrcoef(y_true, y_pred))
    metrics_values['confusion_matrix'].append(confusion_matrix(y_true, y_pred, normalize='all'))

    cr = classification_report(y_true, y_pred, output_dict=True)
    metrics_values['negative_precision'].append(cr['False']['precision'])
    metrics_values['negative_recall'].append(cr['False']['recall'])
    metrics_values['negative_f1'].append(cr['False']['f1-score'])
    metrics_values['positive_precision'].append(cr['True']['precision'])
    metrics_values['positive_recall'].append(cr['True']['recall'])
    metrics_values['positive_f1'].append(cr['True']['f1-score'])
    metrics_values['accuracy'].append(cr['accuracy'])

    loss_fn = BinaryCrossentropy()
    metrics_values['loss'].append(loss_fn(raw_y_true, raw_y_pred).numpy())

Liczona jest średnia metryk.

In [ ]:
#map dict into average values
print("pos support: ", list(raw_y_true).count(1))
print("neg support: ",list(raw_y_true).count(0))
average_metrics_values = {k:np.mean(v, axis=0) for k,v in metrics_values.items()}
for k,v in average_metrics_values.items():
    if(k!='confusion_matrix'):
        v=round(v, 3)
    print(v,"-", k)

Poniżej wyświetli się kilkanaście wykresów oceniających model.

Wykresy Loss od epoki oraz Accuracy od epoki jest stworzony tylko wyłącznie na podstawie ostatniego trenowania, lecz tyle powinno wystarczyć by zobaczyć jak się wykres ogólnie układa.

In [ ]:
# Visualize the training history to see whether you're overfitting.

plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['TRAIN', 'VAL'], loc='lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['Train', 'VAL'], loc='lower right')
plt.show()

In [ ]:
#Plot confusion matrix

labels = ["Straight \n (Neg)", "Slouching \n (Pos)"]
cm=average_metrics_values['confusion_matrix']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot()


Poniżej jest wykres, który pokazuje jak dokładność, MCC i precyzja się zmieniają w zależności od ustawionego progu (thresholdu). Służyło ku zbadaniu najlepszego progu dla programu.

In [ ]:

thresholds = np.arange(0.05, 1, 0.01)
accuracies = []
precisions = []
mccs=[]
for threshold in thresholds:
    y_pred = tf.math.greater(raw_y_pred,threshold)
    y_true = tf.math.greater(raw_y_true,threshold)

    accuracy = accuracy_score(y_true, y_pred)
    accuracies.append(accuracy)

    precision = precision_score(y_true, y_pred)
    precisions.append(precision)

    mcc = matthews_corrcoef(y_true, y_pred)
    mccs.append(mcc)

plt.axvline(x=0.8, color='orange', linestyle='--')
plt.axhline(y=0.936, color='orange', linestyle='--')
plt.plot(thresholds, accuracies, label="accuracy")
plt.plot(thresholds, precisions, label="precision")
plt.plot(thresholds, mccs, label="mcc")
plt.xlabel("Threshold")
plt.title("Accuracy & Precision & MCC against Threshold")
plt.legend()
plt.show()

def display_cm(y_true, y_pred, threshold=0.5, normalize=None):
    labels = ["Straight \n (Neg)", "Slouching \n (Pos)"]
    y_pred = tf.math.greater(y_pred,threshold)
    y_true = tf.math.greater(y_true,threshold)
    cm = confusion_matrix(y_true, y_pred, normalize=normalize)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels,)
    disp.plot()
display_cm(raw_y_true, raw_y_pred, threshold=0.8, normalize='all')


In [ ]:
threshold_tested = 0.8

max_accuracy=round(max(accuracies), 3)
print("Max accuracy: ", max_accuracy)

threshold_0_8_index=np.where(np.round(thresholds,2)==threshold_tested)[0][0]

accuracy_for_tested_threshold=round(accuracies[threshold_0_8_index], 3)
precision_for_tested_threshold=round(precisions[threshold_0_8_index], 3)
print("Tested threshold", threshold_tested)
print("Accuracy for tested threshold ", accuracy_for_tested_threshold)
print("Precision for tested threshold ", precision_for_tested_threshold)

### Zapisywanie modelu: proszę odkomentować poniższy kod w celu zapisania modelu.

In [ ]:
model_save_path='../program_output/classification_model_tf'

In [ ]:
#model.save(model_save_path)